In [ ]:
!pip install opencv-python google-cloud-vision scipy


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.1/39.5 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.2/39.5 MB 1.1 MB/s eta 0:00:35
   ---------------------------------------- 0.2/39.5 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.3/39.5 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.4/39.5 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.4/39.5 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.5/39.5 MB 1.4 MB/s eta 0:00:29
    --------------------------------------- 0.6/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.6/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.7/39.5 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.8/39.5 MB 1.4 MB/s eta 0:00:28
    --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.


## Para Linux

In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS="nimble-root-457808-r2-b639a6729402.json"

'export' is not recognized as an internal or external command,
operable program or batch file.


## Para Windows

In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS="C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json"

env: GOOGLE_APPLICATION_CREDENTIALS="C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json"


In [3]:
import cv2
import io
import os
import numpy as np
from google.cloud import vision
from scipy.spatial import distance as dist # Para calcular distancias euclidianas
from collections import OrderedDict # Para almacenar los objetos rastreados

In [4]:
# detect_people_webcam.py
# Uso: Captura vídeo de la webcam y detecta personas con Google Cloud Vision API.
#
# Requisitos:
#  - Python 3.x
#  - Biblioteca OpenCV y cliente de Vision API:
#      pip install opencv-python google-cloud-vision
#
# Configuración de credenciales de GCP:
#  * En Linux/macOS (bash/zsh):
#      export GOOGLE_APPLICATION_CREDENTIALS="/ruta/a/tu/credencial.json"
#  * En Windows (PowerShell):
#      $env:GOOGLE_APPLICATION_CREDENTIALS = 'C:\ruta\a\tu\credencial.json'
#    o en el Símbolo del sistema (CMD):
#      set GOOGLE_APPLICATION_CREDENTIALS=C:\ruta\a\tu\credencial.json
#  * En Jupyter Notebook:
#      %env GOOGLE_APPLICATION_CREDENTIALS=C:\ruta\a\tu\credencial.json
#
# Opcional: definir la variable de entorno dentro del script (descomenta y ajusta la ruta):
import os
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json'
#
# Asegúrate de habilitar la API Vision y de que la cuenta de servicio tenga permisos de "Cloud Vision API User".

# Si no has configurado la variable externamente, puedes usar la línea siguiente:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json'

class CentroidTracker:
    def __init__(self, maxDisappeared=50, 
                 lost_track_timeout_frames=150, # Frames para recordar una pista perdida (~5 seg a 30fps)
                 hist_comparison_threshold=0.4): # Umbral para comparar histogramas (Bhattacharyya: 0=perfecto, >0=diferente) - ¡NECESITA AJUSTE!
        
        self.nextObjectID = 0
        # Guarda ID -> {'centroid': c, 'bbox': b, 'histogram': h}
        self.objects = OrderedDict() 
        self.disappeared = OrderedDict()
        # Guarda ID -> {'centroid': c, 'bbox': b, 'histogram': h, 'deregistered_frame': frame_num}
        self.lost_tracks = OrderedDict() 

        # --- Parámetros ---
        self.maxDisappeared = maxDisappeared 
        self.lost_track_timeout = lost_track_timeout_frames
        self.hist_threshold = hist_comparison_threshold # Umbral de comparación

    # --- Métodos Auxiliares ---
    def _calculate_histogram(self, roi):
        """Calcula y normaliza un histograma 2D HSV (Hue, Saturation)."""
        if roi is None or roi.size == 0:
            # print("[Advertencia] ROI inválido recibido en _calculate_histogram.")
            return None 
        try:
            hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            # Aumentar los bins puede dar más detalle, pero ser más sensible
            # Bins: Hue(0-179), Saturation(0-255)
            hist = cv2.calcHist([hsv], [0, 1], None, [50, 60], [0, 180, 0, 256])
            # Normalizar para que la comparación sea más estable a cambios de tamaño/iluminación
            cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
            return hist.flatten() # Devolver como vector plano
        except cv2.error as e:
            # print(f"[Error CV2] Calculando histograma: {e}. ROI shape: {roi.shape}")
            return None

    def _purge_lost_tracks(self, current_frame_count):
        """Elimina pistas perdidas que llevan demasiado tiempo desaparecidas."""
        ids_to_purge = []
        for objectID, data in self.lost_tracks.items():
            if current_frame_count - data['deregistered_frame'] > self.lost_track_timeout:
                ids_to_purge.append(objectID)
        
        for objectID in ids_to_purge:
            # print(f"[INFO] Purgando pista perdida ID: {objectID} (timeout)")
            del self.lost_tracks[objectID]

    # --- Métodos Principales ---
    # MODIFICADO: Acepta histograma
    def register(self, centroid, bbox, histogram):
        """Registra un nuevo objeto con su ID, centroide, bbox e histograma."""
        self.objects[self.nextObjectID] = {
            'centroid': centroid, 
            'bbox': bbox, 
            'histogram': histogram
        }
        self.disappeared[self.nextObjectID] = 0
        # print(f"[INFO] Registrando nuevo objeto ID: {self.nextObjectID}")
        current_id = self.nextObjectID
        self.nextObjectID += 1
        return current_id # Devuelve el ID asignado

    # MODIFICADO: Mueve a lost_tracks en lugar de borrar directamente
    def deregister(self, objectID, current_frame_count):
        """Mueve un objeto a 'lost_tracks' cuando desaparece."""
        if objectID in self.objects:
            # print(f"[INFO] Moviendo objeto ID {objectID} a lost tracks.")
            # Guarda la información relevante en lost_tracks
            self.lost_tracks[objectID] = {
                'centroid': self.objects[objectID]['centroid'],
                'bbox': self.objects[objectID]['bbox'],
                'histogram': self.objects[objectID]['histogram'],
                'deregistered_frame': current_frame_count 
            }
            # Elimina de las listas activas
            del self.objects[objectID]
            if objectID in self.disappeared:
                del self.disappeared[objectID]
        # else:
            # print(f"[Advertencia] Intento de desregistrar ID {objectID} que no está en self.objects.")


    # MODIFICADO: Acepta ROIs y frame_count, incluye lógica ReID
    def update(self, rects, rois, current_frame_count):
        """Actualiza el estado del tracker, intentando ReID con histogramas."""
        
        # 0. Purga pistas perdidas antiguas
        self._purge_lost_tracks(current_frame_count)

        # 1. Manejar caso sin detecciones actuales
        if len(rects) == 0:
            ids_a_eliminar = []
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    ids_a_eliminar.append(objectID)
            # Mueve los desaparecidos a lost_tracks
            for objectID in ids_a_eliminar:
                 self.deregister(objectID, current_frame_count)
            return self.objects # Devuelve objetos activos (posiblemente vacío)

        # 2. Preparar centroides, bboxes e histogramas de entrada
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        inputBBoxes = [None] * len(rects)
        inputHistograms = [None] * len(rects)
        valid_indices = [] # Indices de rects/rois que son válidos
        
        for i in range(len(rects)):
             startX, startY, endX, endY = rects[i]
             # Validar ROI antes de calcular histograma
             if i < len(rois) and rois[i] is not None and rois[i].size > 0:
                hist = self._calculate_histogram(rois[i])
                if hist is not None:
                     inputHistograms[i] = hist
                     # Calcular centroide solo si el histograma es válido
                     cX = int((startX + endX) / 2.0)
                     cY = int((startY + endY) / 2.0)
                     inputCentroids[i] = (cX, cY)
                     inputBBoxes[i] = rects[i]
                     valid_indices.append(i)
                # else:
                     # print(f"[Advertencia] Histograma nulo para ROI {i}, saltando rect.")
             # else:
                 # print(f"[Advertencia] ROI inválido o faltante para rect {i}, saltando.")

        # Filtrar entradas inválidas (si alguna ROI falló)
        if len(valid_indices) != len(rects):
             # print(f"[INFO] Filtrando {len(rects) - len(valid_indices)} detecciones por ROI/histograma inválido.")
             inputCentroids = inputCentroids[valid_indices]
             inputBBoxes = [inputBBoxes[i] for i in valid_indices]
             inputHistograms = [inputHistograms[i] for i in valid_indices]
             # Si no queda ninguna detección válida
             if not valid_indices:
                 # Tratar como si no hubiera detecciones (paso 1 se aplica a objetos existentes)
                 return self.update([], [], current_frame_count)


        # 3. Manejar caso inicial (sin objetos rastreados activos)
        if len(self.objects) == 0:
            for i in range(len(inputCentroids)):
                # Antes de registrar como nuevo, intenta ReID con lost_tracks
                reidentified_id = self._attempt_reid(inputHistograms[i])
                if reidentified_id != -1:
                     # print(f"[INFO] Re-identificado! Nueva detección coincide con ID perdido {reidentified_id}.")
                     self.objects[reidentified_id] = {
                        'centroid': inputCentroids[i], 
                        'bbox': inputBBoxes[i], 
                        'histogram': inputHistograms[i]
                     }
                     self.disappeared[reidentified_id] = 0
                     del self.lost_tracks[reidentified_id] # Eliminar de perdidos
                else:
                     # No hubo ReID, registrar como nuevo
                     self.register(inputCentroids[i], inputBBoxes[i], inputHistograms[i])
            return self.objects # Salir temprano

        # 4. Emparejamiento posicional con objetos activos
        objectIDs = list(self.objects.keys())
        objectCentroids = [data['centroid'] for data in self.objects.values()]
        
        # Calcular distancias entre centroides activos y nuevos
        D = dist.cdist(np.array(objectCentroids), inputCentroids)
        rows = D.min(axis=1).argsort()
        cols = D.argmin(axis=1)[rows]

        usedRows = set() # Indices de objetos activos ya emparejados
        usedCols = set() # Indices de nuevas detecciones ya emparejadas

        for (row, col) in zip(rows, cols):
            if row in usedRows or col in usedCols: continue
            
            # --- Opcional: Umbral de distancia posicional ---
            # distance_threshold = 75 
            # if D[row, col] > distance_threshold: continue
            # ---------------------------------------------

            objectID = objectIDs[row]
            self.objects[objectID]['centroid'] = inputCentroids[col]
            self.objects[objectID]['bbox'] = inputBBoxes[col]
            # Actualizar histograma también (opcional, ¿o mantener el original?)
            # Mantener el original podría ser más estable si la detección actual es ruidosa
            # O actualizarlo si la apariencia cambia gradualmente:
            self.objects[objectID]['histogram'] = inputHistograms[col] 
            self.disappeared[objectID] = 0
            usedRows.add(row)
            usedCols.add(col)

        # 5. Manejar objetos activos no emparejados (posiblemente desaparecidos)
        unusedRows = set(range(len(objectCentroids))).difference(usedRows)
        for row in unusedRows:
            objectID = objectIDs[row]
            self.disappeared[objectID] += 1
            if self.disappeared[objectID] > self.maxDisappeared:
                self.deregister(objectID, current_frame_count) # Mover a lost_tracks

        # 6. Manejar nuevas detecciones no emparejadas (intentar ReID, luego registrar)
        unusedCols = set(range(len(inputCentroids))).difference(usedCols)
        
        registered_new_ids = [] # Guardar los IDs nuevos asignados en este frame
        for col in unusedCols:
             # Intentar ReID ANTES de registrar como nuevo
             reidentified_id = self._attempt_reid(inputHistograms[col])
             if reidentified_id != -1:
                 # print(f"[INFO] Re-identificado! Detección {col} coincide con ID perdido {reidentified_id}.")
                 # Reactivar el ID perdido con la nueva información
                 self.objects[reidentified_id] = {
                    'centroid': inputCentroids[col], 
                    'bbox': inputBBoxes[col], 
                    'histogram': inputHistograms[col] # Usar el nuevo histograma
                 }
                 self.disappeared[reidentified_id] = 0
                 del self.lost_tracks[reidentified_id] # Eliminar de perdidos
             else:
                 # No hubo ReID, registrar como completamente nuevo
                 new_id = self.register(inputCentroids[col], inputBBoxes[col], inputHistograms[col])
                 registered_new_ids.append(new_id)

        # print(f"[Debug] Nuevos IDs registrados en este frame: {registered_new_ids}")
        return self.objects

    def _attempt_reid(self, new_histogram):
        """Compara un histograma nuevo con los de lost_tracks."""
        best_match_id = -1
        # Usar el umbral definido en __init__
        min_dist = self.hist_threshold 

        if new_histogram is None: # No se puede comparar si no hay histograma
             return -1

        for lost_id, lost_data in self.lost_tracks.items():
            lost_hist = lost_data.get('histogram')
            if lost_hist is None: continue # Saltar si la pista perdida no tiene histograma

            try:
                # Comparar usando distancia Bhattacharyya (más robusta para histogramas)
                dist = cv2.compareHist(new_histogram, lost_hist, cv2.HISTCMP_BHATTACHARYYA)
                # print(f"[Debug ReID] Comparando con ID perdido {lost_id}: Distancia = {dist:.4f}")
                
                # Si la distancia es menor que la mínima encontrada HASTA AHORA
                # y menor que el UMBRAL definido
                if dist < min_dist:
                    min_dist = dist
                    best_match_id = lost_id
            except cv2.error as e:
                # print(f"[Error CV2] Comparando histogramas: {e}")
                continue # Saltar esta comparación si falla

        if best_match_id != -1:
            # print(f"[Debug ReID] Mejor coincidencia encontrada: ID {best_match_id} con distancia {min_dist:.4f}")
            pass

        return best_match_id

# --- Función Principal (MODIFICADA para extraer ROIs y llamar a update con ellas) ---
def main():
    # ... (Inicialización cliente API - sin cambios) ...
    try:
        client = vision.ImageAnnotatorClient()
        print("Cliente de Vision API inicializado correctamente.")
    except Exception as e:
        print(f"Error al inicializar el cliente de Vision API: {e}")
        print("Asegúrate de que las credenciales de GCP estén configuradas.")
        return

    # Ajustar parámetros del tracker si es necesario
    ct = CentroidTracker(maxDisappeared=50, 
                         lost_track_timeout_frames=150, # ~5 segundos
                         hist_comparison_threshold=0.4) # <-- ¡AJUSTA ESTE UMBRAL! (0.3-0.6?)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se pudo abrir la webcam.")
        return

    print("Iniciando detección. Presiona 'q' para salir.")
    frame_count = 0 

    nms_confidence_threshold = 0.4 
    nms_overlap_threshold = 0.3   

    while True:
        ret, frame = cap.read()
        if not ret: break
        
        frame_count += 1
        (h, w) = frame.shape[:2]
        
        boxes_for_nms = []      
        confidences_for_nms = [] 
        original_rects_pre_nms = [] # Guardar rects antes de NMS

        # ... (Codificación y llamada a la API - sin cambios) ...
        ret2, buf = cv2.imencode('.jpg', frame)
        if not ret2: continue 
        content = buf.tobytes()
        image = vision.Image(content=content)
        try:
            response = client.object_localization(image=image)
            objects_detected = response.localized_object_annotations
            if response.error.message: objects_detected = []
        except Exception as e: objects_detected = []

        # --- Extracción PRE-NMS ---
        for obj in objects_detected:
            if obj.name.lower() == 'person' and obj.score >= nms_confidence_threshold:
                # ... (código para calcular x_min, y_min, x_max, y_max - sin cambios) ...
                verts = obj.bounding_poly.normalized_vertices
                pts_norm = [(max(0.0, min(1.0, v.x)), max(0.0, min(1.0, v.y))) for v in verts]
                pts = [(int(v[0] * w), int(v[1] * h)) for v in pts_norm]
                x_coords = [p[0] for p in pts]; y_coords = [p[1] for p in pts]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                x_min, y_min = max(0, x_min), max(0, y_min)
                x_max, y_max = min(w - 1, x_max), min(h - 1, y_max)

                if x_max > x_min and y_max > y_min:
                    boxes_for_nms.append([x_min, y_min, x_max - x_min, y_max - y_min]) 
                    confidences_for_nms.append(float(obj.score))
                    original_rects_pre_nms.append((x_min, y_min, x_max, y_max)) 

        # --- Aplicar NMS ---
        indices_to_keep = cv2.dnn.NMSBoxes(boxes_for_nms, confidences_for_nms, 
                                             nms_confidence_threshold, nms_overlap_threshold)

        # --- Preparar Rects y ROIs FINALES para el Tracker ---
        rects_after_nms = []
        rois_after_nms = []
        if len(indices_to_keep) > 0:
            if isinstance(indices_to_keep, np.ndarray) and indices_to_keep.ndim > 1:
                 indices_to_keep = indices_to_keep.flatten()
            
            for i in indices_to_keep:
                 if 0 <= i < len(original_rects_pre_nms):
                    (x_min, y_min, x_max, y_max) = original_rects_pre_nms[i]
                    # Extraer ROI del frame ACTUAL
                    roi_y_min, roi_y_max = max(0, y_min), min(h, y_max)
                    roi_x_min, roi_x_max = max(0, x_min), min(w, x_max)
                    
                    if roi_y_max > roi_y_min and roi_x_max > roi_x_min: # Validar tamaño ROI
                        roi = frame[roi_y_min:roi_y_max, roi_x_min:roi_x_max]
                        # Comprobar si el ROI extraído no está vacío (importante!)
                        if roi.size > 0:
                             rects_after_nms.append(original_rects_pre_nms[i])
                             rois_after_nms.append(roi)
                        # else:
                             # print(f"[Advertencia] ROI extraído vacío para rect {original_rects_pre_nms[i]}, saltando.")
                    # else:
                         # print(f"[Advertencia] Coordenadas ROI inválidas para rect {original_rects_pre_nms[i]}, saltando.")
                 # else:
                    # print(f"[Advertencia] Índice NMS inválido: {i}")


        # --- Actualiza el rastreador CON RECTS, ROIs y FRAME COUNT ---
        tracked_people = ct.update(rects_after_nms, rois_after_nms, frame_count) 

        # --- Dibuja los resultados (sin cambios) ---
        for objectID, data in tracked_people.items():
            # ... (código de dibujo - sin cambios) ...
            centroid = data['centroid']
            (startX, startY, endX, endY) = data['bbox'] 
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            text = f"ID {objectID}"
            text_y = startY - 10 if startY - 10 > 10 else startY + 15 
            cv2.putText(frame, text, (startX, text_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2) 

        cv2.imshow('ReID Básico con Histogramas (Presiona q para salir)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()
    print("Recursos liberados. Aplicación finalizada.")


In [5]:
if __name__ == '__main__':
    main()

Cliente de Vision API inicializado correctamente.
Iniciando detección. Presiona 'q' para salir.
Recursos liberados. Aplicación finalizada.
